In [1]:
from packages.display.core import *
%pylab inline
pylab.rcParams['figure.figsize'] = (200, 5)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [31]:
%qtconsole

In [3]:
cube_params = {
  'alpha'    : 0,
  'delta'    : 0,
  'spe_bw'   : 4000,
  'spe_res'  : 1,
  's_f'      : 8,
  's_a'      : 1}

In [4]:
freq_range = np.arange(604000, 718000, 4000)

dbpath = 'ASYDO'
dba = packages.asydopy.db.lineDB(dbpath)
dba.connect()

cube_name = 'data/cube_'

confusion_matrix = []
results = []

In [21]:
for i in [28]:
    f = freq_range[i]
    freq_init = f - 2000
    freq_end = f + 2000
    cube_params['freq'] = f
    
    file_path = " " + str(freq_init) + " - " + str(freq_end)
    
    dictionary = load_dictionary(file_path)
    
    cube_path = 'data/cube_' + file_path + '.fits'
    train_pixel = (1, 1)
    
    dictionary_recal, detected_peaks = recal_words(cube_path, dictionary, cube_params, 100)
    
    X = get_values_filtered_normalized(cube_path, train_pixel, cube_params)

    y_train = get_fortran_array(np.asmatrix(X))
    dictionary_recal_fa = np.asfortranarray(dictionary_recal,
                                            dtype= np.double)
    
    lambda_param = 0
    for idx in range(0, len(detected_peaks)):
        if detected_peaks[idx] != 0:
            lambda_param += 1
    
    param = {
      'lambda1' : lambda_param,
      # 'L': 1,
      'pos' : True,
      'mode' : 0,
      'ols' : True,
      'numThreads' : -1}
    
    alpha = spams.lasso(y_train, dictionary_recal_fa, **param).toarray()
    total = np.inner(dictionary_recal_fa, alpha.T)
    
    if i == 0:
        confusion_matrix = [get_confusion_matrix(dictionary_recal, alpha,
                                   cube_path, cube_params, True)]
        results = [get_results(confusion_matrix[i])]
    else:
        confusion_matrix.append(get_confusion_matrix(dictionary_recal, alpha,
                                               cube_path, cube_params, True))
        results.append(get_results(confusion_matrix[i]))
    print(i)
    


IndexError: list index out of range

In [ ]:
np.mean([np.mean(means["F-Score"]) for means in results])